## Final Outcome: 
1. How many cell-types are in your dataset?
2. Which marker genes characterize the cell-types?
3. What cell-types are likely present?

#### Make sure to include:
1. Dataset details (nb cells, nb detected genes, qc plots such as boxplot of avg. nb of genes expressed per cell, …)
2. Results (answering the main questions, include standard plots to visualize your results (t-SNE, UMAP, clustering, expression of marker genes, …)

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from scipy.stats import median_abs_deviation
import warnings
warnings.filterwarnings("ignore")

## Importing data

In [ ]:
sc.settings.verbosity = 3

df = pd.read_csv("../7_ovary.tsv",sep="\t",header=None)
df.set_index(df.columns[0],inplace=True)

adata = sc.AnnData(df.T)
adata

the dataset has the shape barcodes (cells) x number of transcripts

## Quality Control Analysis
#### QC = Quality control analysis
1. The number of counts per barcode (count depth)
2. The number of genes per barcode 
3. THe fraction of counts from mitochondrial genes per barcode. 


Low quality cells might show a low count depth, few detected genes and a high fraction of mitochondrial reads. Important to consider 3 QC covariates jointly. Similar to [Germain et al., 2020], we mark cells as outliers if they differ by 5 MADs which is a relatively permissive filtering strategy. We want to highlight that it might be reasonable to re-assess the filtering after annotation of cells. 

 Usually, studies us data-agnostic QC filters, set at 5-10% for fraction of mitochondrial reads, 500 for gene complexity. 

Ref: 
1. Subramanian, A., Alperovich, M., Yang, Y. et al. Biology-inspired data-driven quality control for scientific discovery in single-cell transcriptomics. Genome Biol 23, 267 (2022). https://doi.org/10.1186/s13059-022-02820-w
2. https://www.sc-best-practices.org


In [ ]:
#QC 
adata.var['mt'] = adata.var_names.str.startswith('mt:')  # annotate the group of mitochondrial genes as 'mt'
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo'], percent_top=[20], log1p=True, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

1. n_genes_by_counts: number of genes with positive counts in a cell
2. total_counts: total number of counts for a cell = library size 
3. pct_counts_mt: proportion of total counts for a cell which are mitochondrial 

In [ ]:
sns.displot(adata.obs["total_counts"], bins=100, kde=False)

Power-law distribution. The majority of cells have <2500 counts 

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

Low complexity of genes and high mitochondrial read content have been used as a proxy for identifying poor-quality cells . High gene complexity has been used as a proxy for doublets or multiplets in droplet-based sequencing.

### Identifiying Outliers

Because we don't want to filter out too much, we first perform a basic qc with high quality cells defined as >= 100 genes, >= 3 cells <= 80% mito. We will calculate median absolute deviation (MAD) based thresholds after dimensionality reduction and clustering for each cluster

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)
adata = adata[adata.obs["pct_counts_mt"] <= 1, :]

sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

### Normalization 
Normalize the feature expression measurements for each cell by the total expression, multiply by a scale factor (10,000) and log-transfrom the result to get log(TPX + 1) values

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=2000)
sc.pl.highly_variable_genes(adata)

insert analysis of the plot

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

insert analysis of the plot

## Dimension Reduction

### PCA

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True)
sc.pl.pca(adata,color=['log1p_total_counts','log1p_total_counts_mt'])

sc.pl.pca_variance_ratio(adata, log=True)

### UMAP

In [ ]:
sc.pp.neighbors(adata)

sc.tl.umap(adata)
sc.pl.umap(adata, color=['log1p_total_counts','log1p_n_genes_by_counts','pct_counts_mt'])

### Clustering

In [ ]:
sc.tl.leiden(adata, key_added="leiden_res0_5", resolution=0.5)
sc.tl.leiden(adata, key_added="leiden_res1", resolution=1.0)
sc.tl.leiden(adata, key_added="leiden_res2", resolution=2)

sc.pl.umap(adata, color=["leiden_res0_5", "leiden_res1","leiden_res2"],legend_loc="on data",)

QC on clusters 

## Cell annotation

### Differential expression analysis

In [ ]:
#calculate the differentially expressed genes for every cluster, compared to the rest of the cells in our adata 
sc.tl.rank_genes_groups(adata, groupby="leiden_res1", method="t-test", key_added="dea_leiden_1")

#Visualize:
sc.pl.rank_genes_groups_dotplot(adata, groupby="leiden_res1", standard_scale="var", n_genes=5, key="dea_leiden_1")

In [ ]:
# filter the differentially expressed genes to select for more cluster-specific differentially expressed genes:
sc.tl.filter_rank_genes_groups(adata, key="dea_leiden_1", min_in_group_fraction=0.2, max_out_group_fraction=0.2, key_added="dea_leiden_1_filtered")

#Visualise: 
sc.pl.rank_genes_groups_dotplot(adata, groupby="leiden_res1", standard_scale="var", n_genes=5, key="dea_leiden_1_filtered")

In [ ]:
markers = pd.DataFrame(adata.uns["dea_leiden_1_filtered"]["names"])

### Identify cell types
Use the Fly Cell Atlas ovaries dataset: https://flycellatlas.ds.czbiohub.org/ovary/

In [ ]:
for col in markers.columns :
    markers_genes = markers[col].dropna().tolist()[:10]
    sc.pl.umap(adata, color=[j for j in markers_genes ])

nbr types
marker genes characterizing
